Förklaring från Alexei om val av att ha json i snowflake:

Grejen är att metering_point och meter_reading data kan uppdateras i efterhand. T.ex. det finns c1-c5 koefficienterna som kan tillkomma, väderdata, standardavvikelse på mätningen, etc. Problemet är att detta kan hända rätt ofta i fortsättningen och update operationen är tidskrävande. Samtidigt kan datamodellen ändras, osannolikt men möjligt. Då skulle man behöva designa om tabellerna. Därför valde vi att lagra allt data som kommer istället och skapa endast de viktigaste kolumnerna som behövs för att skapa views. Du kan fortfarande köra queries mot json som om det vore SQL. Det är fördelen med snowflake - https://docs.snowflake.com/en/user-guide/querying-semistructured.html

Hämta all öresundskraft data för property=energi, formattera med rätt dataformat direkt i sql

In [2]:
import pyodbc

#Using connection string to connect to the datalake
connection = pyodbc.connect("Driver={SnowflakeDSIIDriver}; Server=lq42418.eu-west-1.snowflakecomputing.com; Database=SMARTENERGI_DATALAKE; schema=public; UID=smartenergi_public_user; PWD=SmartEnergi2020!")
cus=connection.cursor()

#Executing the query on meter readings view
cus.execute("use warehouse compute_wh;")
select_stmt = """SELECT date(SRC:timestamp::datetime) as timestamp, SRC:metering_point_id::string as metering_point_id, sum(SRC:value::integer) as value, SRC:unit_of_measure::string as unit_of_measure
FROM \"SMARTENERGI_DATALAKE\".\"PUBLIC\".\"METER_READINGS\" 
WHERE PROPERTY = \'energy\' and SRC:source_instance_name like \'%oresundskraft%\'
group by date(SRC:timestamp::datetime), SRC:metering_point_id, SRC:unit_of_measure"""
        
cus.execute(select_stmt)


Hämta data

In [3]:
data = cus.fetchall()

Hämta direkt som pandas data frame

In [6]:
import pandas as pd
df = pd.read_sql(select_stmt,connection)

In [7]:
df.head()

,TIMESTAMP,METERING_POINT_ID,VALUE,UNIT_OF_MEASURE
0,2014-02-15,b0455402-7070-11ea-ae00-062fc13530dc,97.0,MWh
1,2014-02-09,b0455402-7070-11ea-ae00-062fc13530dc,114.0,MWh
2,2017-02-26,7dc01c3c-7071-11ea-ae00-062fc13530dc,59.0,MWh
3,2017-04-10,7dc01c3c-7071-11ea-ae00-062fc13530dc,44.0,MWh
4,2017-05-11,7dc01c3c-7071-11ea-ae00-062fc13530dc,45.0,MWh
